In [17]:
### Podcast Micro-Categories
### Unsupervised LDA

from __future__ import print_function
from __future__ import division

import gc
import os
import time

import numpy as np
import pandas as pd

import lda


In [15]:
# I. LOAD ---------------------------------------------------

eps = pd.read_pickle('../interim/028_preproc_heavy_eps.p')

print("Episode Table Shape:", eps.shape)

Episode Table Shape: (1202, 4)


In [16]:
eps.head()


,podcast_name,subgenre,description,description_tokens
0,''A Moment of Yoga - Yoga Podcast,Self-Help,A Moment of Yoga - The Yamas - the restriants...,"[moment, yoga, yama, restriants, raja, yoga, m..."
1,10 English Idioms in 10 Minutes!,K-12,Join Matt Purland as he shares an important an...,"[join, matt, purland, share, important, announ..."
2,100 Days for Good,Non-Profit,"Thanks to great listener feedback, we take ano...","[thanks, great, listener, feedback, take, anot..."
3,10x Talk with Joe Polish and Dan Sullivan,Management & Marketing,"In this Episode of 10X Talk, youll Discover 7 ...","[episode, x, talk, youll, discover, success, h..."
4,1UP.com - The Sports Game Guy's Sports Anomaly,Video Games,4th String borrows from NFL Films to explain t...,"[th, string, borrows, nfl, film, explain, grav..."
